<a href="https://colab.research.google.com/github/jjschueder/7331DataMiningNotebooks/blob/master/lab1/LAB01-DataMining-Joe%20Copy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1

 ### Team:  
Joseph Schueder,
Armando Vela,
Daniel Clark,
Jeff Washburn

##Business Understanding

Introduction - The Iowa Liquor Sales dataset is an API from Google’s Bigquery which contains the wholesale purchases by retail stores in the Iowa area.  The dataset includes the spirit purchase details by product, date of purchase, and location the item was purchased from an Iowa Class “E” liquor license holder (retail stores) . The timeframe of this data starts from January 1, 2012 through 2019. As part of the study commissioned by the Iowa Department of Commerce, all alcoholic sales within the state were logged into the Department system, and in turn, published as open data by the State of Iowa. The dataset contains detail on the name, product, quantity and location of the individual container or package sale between the wholesaler (vendor) and the retailer.  

1. Set Objectives: We are a whiskey wholesaler and we have targeted the state of Iowa for our market analysis of what opportunities are to sell to retailers around the state. Before doing so, our goal will be to identify and predict what opportunities there are in the Iowa liquor market.  Questions will include, which categories of alcholol are going to sell the most in each market, what parts of the year are going to have higher whiskey sales as opposed to lower whiskey sales to optimize advertising dollars, and what features of a town are defining characteristics of a large whiskey seller. 

2. Product Project Plan: To meet our goals, we will first use the Google Bigquery API to access the publicly available Iowa Liquor Sales data via python and Jupyter Notebook. Since the dataset is over 17 million lines of data, we will focus 2019 sales the for this analysis. From here, we will define a random subset to grant us 400,000 observations, address missing values and begin to build models that could help us answer questions based on markets likely to sell more whiskey. For our data on times of the year when we will sell more whiskey, we will need to create a subset of a full year’s data to build a time series model. For the town features, we will need to include additional information about the zip codes of Iowa (such as population, income, ect) to get more demographic detail on each market to assist with a program for a clustering problem. For each of these problems, we will create a training and testing data set to help tune our predictions. 

3. Business Success Criteria - For our prediction models of market sales, we will consider our model successful if we are able to classify our alcohol type with a strong precision, recall, accuracy,  and F1 score using a cross validation. Ideally, if we are dealing with eight alcohol types, we will want to be able to predict whiskey accurately over 12.5% of the time.  
 * Classification - 'Introduction to Data Mining' book - pages 173 to 196.  Estimating the generalization error of a model during training. The estimated error helps the learning algorithm to do model selection; i.e., to find a model of the right complexity that is not susceptible to over fitting. Once the model bas been constructed, it can be applied to the test set to predict the class labels of previously unseen records. It is often useful to measure the performance of the model on the test set because such a measure provides an unbiased estimate of its generalization error. The accuracy or error rate computed from the test set can also be used to compare the relative performance of different classifiers on the same domain. However, is order to do this, the class labels of the test records must be known.  Some of the methods commonly used to evaluate the performance of a classifier are.
    * Holdout Method In the holdout method, the original data with labeled examples is partitioned into two disjoint sets, called the training and the test sets, respectively. A classification model is then induced from the training set and its performance is evaluated on the test set. The proportion of data reserved for training and for testing is typically at the discretion of the analyst (e.g., 50-50 or two-thirds for training and one-third for testing). The accuracy of the classifier can be estimated based on the accuracy of the induced model on the test set. 
     * Other methods that may be used Cross-Validation, Random subsampling, and bootstrap. 
 
 * Regression - From the sales prediction problem, we will run a linear regression technique against the remaining features. For this technique, we will use Root Mean Square Error to evaluate the effectiveness. A strong criteria for performance would be if we can get within a certain RMSE. We may also use R-squared as a relative measure of fit and OLS. Reference to SMU MSDS class 6371 Units 9 through 11 will be used for evlaution methods. Lack of fit tests, scatterplot of residual, studentized residuals and other fit diagnostics will all be utilized. 
 


## Data Meaning Type

The initial data set is 4.63GB with 17.7 million rows.  This was too large for our systems to handle, so we took a subset of the data.  Please refer to the Exceptional Work on how we analyzed the full data set, which included a combination of BigQuery queries, and visualizing that data in order to come up with our subset which we saved as a csv file in our github repository

### Load Python Packages

In [0]:
# Import necessary python packages
try:
    from collections import abc as collections_abc
except ImportError:  # Python 2.7
    import collections as collections_abc

    import copy
import functools
import gzip
import io
import itertools
import json
import math
import os
import tempfile
import uuid
import warnings
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import numpy as np
#import altair as alt
import matplotlib.pyplot as plt
import re
import warnings
warnings.simplefilter('ignore', DeprecationWarning)

# Imports the Google Cloud client library
#from google.cloud import storage
from google.oauth2 import service_account
from google.cloud import bigquery

### Load Data from github

So we will be querying the 2019 data and subsetting a random sample of 400,000 rows within the 2 million sales that happened that year for our analysis

In [0]:
# Read csv from disk
#df =  pd.read_csv(r'/Users/jjschued/Documents/Github/7331DataMiningNotebooks/lab1/iowa_subset_2019_400k_random_rows.csv', nrows = 100000)
#df =  pd.read_csv(r'/Users/danielclark/Desktop/SMU/data_mining/7331DataMiningNotebooks/lab1/iowa_subset_2019_400k_random_rows.csv', nrows = 50000)

# read csv from github directly
url_dataset = 'https://github.com/jjschueder/7331DataMiningNotebooks/blob/master/lab1/iowa_subset_2019_400k_random_rows.csv?raw=true'
#df = pd.read_csv(url_dataset, nrows=50000)
df = pd.read_csv(url_dataset)

# verify data read in
df.head(5)

### Columns and Descriptions

Below are the 24 columns found in the data set along with description of each 

In [0]:
df.info()

Detailed Description of each field - referenced from the [data.iowa.gov](https://data.iowa.gov/Sales-Distribution/Iowa-Liquor-Sales/m3tr-qhgy) site that describes the dataset

*   invoice_and_item_number (text) - Invoice and item associated with the liquor order.  This is unique identifier 
*   date (date) - Date of when the order was placed
*   store_number (int) - Unique number assigned to the retail store that placed the order
*   store_name (text) - Name of the retail store that placed the order
*   address (text) - Address of the retail store that placed the order
*   city (text) - City of the retail store that placed the order
*   zip_code (float) - Zip code of the retail store that placed the order
*   store_location (text) - Lat/Long of retail store that placed the order
*   county_number (float) - Iowa county number for which the retail store that placed the order resides
*   county (text) - County name for which the retail store is located that placed the order
*   category (float) - Category code for the liquor that was ordered
*   category_name (text) - Category of the liquor that was ordered
*   vendor_number (int) - The vendor number of the company for the brand of liquor ordered 
*   vendor_name (text) - The vendor name of the company for the brand of liquor ordered
*   item_number (int) - Item number for individual liquor product ordered
*   item_description (text) - Description of the liquor item ordered
*   pack (int) - The number of bottles in a case for the liquor ordered
*   bottle_volume_ml (int) - Volume of each liquor bottle ordered in milliliters
*   state_bottle_cost (float) - The amount that Alcoholic Beverages Division paid for each bottle of liquor ordered
*   state_bottle_retail (float) - The amount the store paid for each bottle of liquor ordered
*   bottles_sold (int) - Number of bottles of liquor ordered by the retail store
*   sale_dollars (float) - Total cost of liquor ordered (bottles_sold * state_bottle_retail)
*   volume_sold_liters (float) - Total volume of liquor ordered in liters (bottle_volume_ml * bottles_sold / 1000)
*   volume_sold_gallons (float) - Total volume of liquor ordered in gallons (bottle_volume_ml * bottles_sold / 3785.411784)





## Data Quality  
Verify data quality: Explain any missing values, duplicate data, and outliers. Are those mistakes? How do you deal with these problems? Give justifications for your methods.

Running a df.columns.values function, confirms the 24 features that we referenced in our Data understanding phase. This now allows us to move forward with our data cleaning. 

In [0]:
df.columns.values

### Data Cleaning

In our data cleaning step, we need to perform a few specific data cleaning operations to:
1.   Convert our features to the correct continuous, ordinal and categorical features
  * Replace the values for pack, bottle_volume_ml, store_number, store_name, address, city, zip_code, county_number, county, category, category_name, vendor_number, vendor_name, item_number and item_description into categorical variables so they register as a non-null object in our model

2.   Address the missing values
  *   The data has a few fields that are missing data. These missing values primarily are centered around location related information and liquor categorization. Some cleanup will be done by referencing external sources with store name and item description to infer address information. 
  *   Replace all "?", which our dataset denotes as null, into "-1" values (not strings). From here, we will convert state_bottle_cost, state_bottle_retail, sale_dollars, volume_sold_liters, and volume_sold_gallons into continuous variables so they register as floats.

3.  Check on duplicates.  
4.  Create a category that simplifies our alcohol categories to specific genres like whiskey, vodka, tequilla, ect.
5.   Categorize our store locations into a few easily discernable buckets
6.   Create a month and date column for opportunities to time and date analysis
7.   Convert the varaible "bottles_sold" into ordinal features so they register as an integer value in our models  
8. Identify outliers and normalize them so that they don't skew our analysis and models."

Using a df.types function helps to verify this. 

In [0]:
df.dtypes

The following will do some cleanup on values and categorize store, category, and dates into more summarized values

In [0]:
#upper case category name for matching later
df['category_name'] = df['category_name'].str.upper()
#df['category_name'][40:45]

In [0]:
#upper case category name for matching later
df['item_description'] = df['item_description'].str.upper()
#df['item_description'][40:45]

In [0]:
#convert nan to blanks
df = (df.replace(r'^\s*$', np.nan, regex=True))
df = (df.replace(np.nan, 'blank', regex=True))

### Missing Values  
The below section will first look for blank address info. Find the Storenames of those blank addresses and finally assign based on external address research. It becomes obvious that the data entry personnel in LeMars, Ankney and Clear lake require retraining.

In [0]:
#blank address info adress info 
df['counter'] = df['sale_dollars']/df['sale_dollars']
dfc = df.copy()
dfc = dfc[dfc['address']=='blank']
df_grouped = dfc.groupby(by=['store_name'])
sales_rateq = df_grouped.counter.sum()
dfsr = pd.DataFrame(sales_rateq)

In [0]:
dfsr.sort_values(by='counter', ascending=False).head(20)

In [0]:
df.loc[df['store_name'] =='Hy-Vee Food Store #2 / State Ankeny', 'address'] = '2510 SW State St'
df.loc[df['store_name'] =='Hy-Vee Food Store #2 / State Ankeny', 'city']	= 'Ankeny'
df.loc[df['store_name'] =='Hy-Vee Food Store #2 / State Ankeny', 'zip_code']	= '50023'
df.loc[df['store_name'] =='Hy-Vee Food Store #2 / State Ankeny', 'store_location'] ='POINT (-93.621824 41.705188)'
df.loc[df['store_name'] =='Hy-Vee Food Store #2 / State Ankeny', 'county_number'] ='77'
df.loc[df['store_name'] =='Hy-Vee Food Store #2 / State Ankeny', 'county'] ='POLK'
df.loc[df['store_name'] == 'Quick Shop / Clear Lake', 'address'] = '904 N 8th St'	
df.loc[df['store_name'] == 'Quick Shop / Clear Lake','city']	= 'Clear Lake'	
df.loc[df['store_name'] == 'Quick Shop / Clear Lake', 'zip_code']	= '50428'	

df.loc[df['store_name'] == 'Quick Shop / Clear Lake', 'store_location'] ='POINT (-93.378772 43.142868)'	
df.loc[df['store_name'] == 'Quick Shop / Clear Lake', 'county_number'] ='17'
df.loc[df['store_name'] == 'Quick Shop / Clear Lake', 'county']= 'Cerro Gord'

df.loc[df['store_name'] == 'Lake Liquors Wine and Spirits', 'address'] = '910 Hwy 18 W'
df.loc[df['store_name'] == 'Lake Liquors Wine and Spirits', 'city']	= 'Clear Lake'	
df.loc[df['store_name'] == 'Lake Liquors Wine and Spirits', 'zip_code'] = '50428'	
df.loc[df['store_name'] == 'Lake Liquors Wine and Spirits', 'store_location'] ='POINT (-93.613385 43.105949)'	
df.loc[df['store_name'] == 'Lake Liquors Wine and Spirits', 'county_number'] ='17'	
df.loc[df['store_name'] == 'Lake Liquors Wine and Spirits', 'county'] = 'CERRO GORD'

df.loc[df['store_name'] == 'Hy-Vee Wine and Spirits / Algona', 'address'] = '1516 Highway 169'
df.loc[df['store_name'] == 'Hy-Vee Wine and Spirits / Algona', 'city']	= 'North Algona'	
df.loc[df['store_name'] == 'Hy-Vee Wine and Spirits / Algona', 'zip_code']= '50511'	
df.loc[df['store_name'] == 'Hy-Vee Wine and Spirits / Algona', 'store_location'] ='POINT (-94.236003 43.081288)'	
df.loc[df['store_name'] == 'Hy-Vee Wine and Spirits / Algona', 'county_number'] ='55'	
df.loc[df['store_name'] == 'Hy-Vee Wine and Spirits / Algona', 'county'] = 'KOSSUTH'

df.loc[df['store_name'] == 'Hy-Vee Wine and Spirits / Lemars', 'address'] = '1201 12th Ave SW'	
df.loc[df['store_name'] == 'Hy-Vee Wine and Spirits / Lemars', 'city']	= 'Le Mars'
df.loc[df['store_name'] == 'Hy-Vee Wine and Spirits / Lemars', 'zip_code']	= '51031'	
df.loc[df['store_name'] == 'Hy-Vee Wine and Spirits / Lemars', 'store_location'] = 'POINT (-96.18335000000002 42.778257)'	
df.loc[df['store_name'] == 'Hy-Vee Wine and Spirits / Lemars', 'county_number'] = '75'	
df.loc[df['store_name'] == 'Hy-Vee Wine and Spirits / Lemars', 'county'] = 'PLYMOUTH'


df.loc[df['store_name'] == 'The Music Station', 'address'] = '2001 West Court'
df.loc[df['store_name'] == 'Hometown Foods / Stuart', 'city']	= 'Cedar Falls'
df.loc[df['store_name'] == 'Hometown Foods / Stuart', 'zip_code'] = '50647'	
df.loc[df['store_name'] == 'Hometown Foods / Stuart', 'store_location'] ='POINT (-92.462826 42.537839)'	
df.loc[df['store_name'] == 'Hometown Foods / Stuart', 'county_number'] ='7'	
df.loc[df['store_name'] == 'Hometown Foods / Stuart', 'county'] = 'BLACK HAWK'

df.loc[df['store_name'] == 'Hometown Foods / Stuart', 'address'] = '611 S Division St'
df.loc[df['store_name'] == 'Hometown Foods / Stuart', 'city']	= 'Stuart'	
df.loc[df['store_name'] == 'Hometown Foods / Stuart', 'zip_code'] = '50250'	
df.loc[df['store_name'] == 'Hometown Foods / Stuart', 'store_location'] ='POINT (-94.318443 41.49759900000001)'
df.loc[df['store_name'] == 'Hometown Foods / Stuart', 'county_number'] ='1'	
df.loc[df['store_name'] == 'Hometown Foods / Stuart', 'county'] = 'Adair'

The below section will first look for blank liquor category info. Find the Item Descriptions of those blank categories and finally assign based on external market research.

In [0]:
#blank  info category info 
df['counter'] = df['sale_dollars']/df['sale_dollars']
dfc = df.copy()
dfc = dfc[dfc['category_name']=='blank']
df_grouped = dfc.groupby(by=['item_description'])
sales_rateq = df_grouped.counter.sum()
dfsr = pd.DataFrame(sales_rateq)

In [0]:
dfsr.sort_values(by='counter', ascending=False).head(10)

In [0]:
#do the cleanup
df.loc[df['category_name'].str.contains('blank') & df['item_description'].str.contains('99'), 'category_name'] = 'American Vodkas'
df.loc[df['category_name'].str.contains('blank') & df['item_description'].str.contains('Eddy'), 'category_name'] = 'American Vodkas'
df.loc[df['category_name'].str.contains('blank') & df['item_description'].str.contains('Rum'), 'category_name'] = 'Flavored Rum'
df.loc[df['category_name'].str.contains('blank') & df['item_description'].str.contains('Liquer'), 'category_name'] = 'American Cordials and Liquers'
df.loc[df['category_name'].str.contains('blank') & df['item_description'].str.contains('Bourbon'), 'category_name'] = 'Straight Rye Whiskies'
df.loc[df['category_name'].str.contains('blank') & df['item_description'].str.contains('Ever'), 'category_name'] = 'Neutral Grain Spirits'

In [0]:
#Start with everything going into Other, and then look for keyword to put into specific category
df['liquor_category'] = 'Other'
df.loc[df['category_name'].str.contains('GINS'), 'liquor_category'] = 'GIN'
df.loc[df['category_name'].str.contains('GINS'), 'liquor_category'] = 'GIN'
df.loc[df['category_name'].str.contains('GIN'), 'liquor_category'] = 'GIN'
df.loc[df['category_name'].str.contains('RUMS'), 'liquor_category'] = 'RUM'
df.loc[df['category_name'].str.contains('RUM'), 'liquor_category'] = 'RUM'
df.loc[df['category_name'].str.contains('SCOTCH'), 'liquor_category'] = 'WHISKY'
df.loc[df['category_name'].str.contains('WHISKIES'), 'liquor_category'] = 'WHISKY'
df.loc[df['category_name'].str.contains('WHISKY'), 'liquor_category'] = 'WHISKY'
df.loc[df['category_name'].str.contains('WHISKEY'), 'liquor_category'] = 'WHISKY'
df.loc[df['category_name'].str.contains('RYE'), 'liquor_category'] = 'WHISKY'
df.loc[df['category_name'].str.contains('AMARETTO'), 'liquor_category'] = 'AMARETTO'
df.loc[df['category_name'].str.contains('BOURBON'), 'liquor_category'] = 'WHISKY'
df.loc[df['category_name'].str.contains('LIQUEURS'), 'liquor_category'] = 'LIQUEUR'
df.loc[df['category_name'].str.contains('LIQUEUR'), 'liquor_category'] = 'LIQUEUR'
df.loc[df['category_name'].str.contains('VODKAS'), 'liquor_category'] = 'VODKA'
df.loc[df['category_name'].str.contains('VODKA'), 'liquor_category'] = 'VODKA'
df.loc[df['category_name'].str.contains('BRANDY'), 'liquor_category'] = 'BRANDY'
df.loc[df['category_name'].str.contains('BRANDIES'), 'liquor_category'] = 'GIN'
df.loc[df['category_name'].str.contains('CREME'), 'liquor_category'] = 'SCHNAPPS'
df.loc[df['category_name'].str.contains('SCHNAPPS'), 'liquor_category'] = 'SCHNAPPS'
df.loc[df['category_name'].str.contains('TEQUILA'), 'liquor_category'] = 'TEQUILA'
df.loc[df['category_name'].str.contains('TEQUILAS'), 'liquor_category'] = 'TEQUILA'



#get a few stray not available categories per internet this is tequila
df.loc[df['item_description'] == 'Herradura Gold Reposado 6pak', 'liquor_category'] = "Tequila"
df.loc[df['item_description'] == 'Chambord Liqueur w/2 Glasses', 'liquor_category'] = "Liquers"



In [0]:
#get some others by going to item description and doing same translation as above
df.loc[df['liquor_category'].str.contains('Other') & df['item_description'].str.contains('GINS'), 'liquor_category'] = 'GIN'
df.loc[df['liquor_category'].str.contains('Other') & df['item_description'].str.contains('GINS'), 'liquor_category'] = 'GIN'
df.loc[df['liquor_category'].str.contains('Other') & df['item_description'].str.contains('GIN'), 'liquor_category'] = 'GIN'
df.loc[df['liquor_category'].str.contains('Other') & df['item_description'].str.contains('RUMS'), 'liquor_category'] = 'RUM'
df.loc[df['liquor_category'].str.contains('Other') & df['item_description'].str.contains('RUM'), 'liquor_category'] = 'RUM'
df.loc[df['liquor_category'].str.contains('Other') & df['item_description'].str.contains('SCOTCH'), 'liquor_category'] = 'WHISKY'
df.loc[df['liquor_category'].str.contains('Other') & df['item_description'].str.contains('WHISKIES'), 'liquor_category'] = 'WHISKY'
df.loc[df['liquor_category'].str.contains('Other') & df['item_description'].str.contains('WHISKY'), 'liquor_category'] = 'WHISKY'
df.loc[df['liquor_category'].str.contains('Other') & df['item_description'].str.contains('WHISKEY'), 'liquor_category'] = 'WHISKY'
df.loc[df['liquor_category'].str.contains('Other') & df['item_description'].str.contains('RYE'), 'liquor_category'] = 'WHISKY'
df.loc[df['liquor_category'].str.contains('Other') & df['item_description'].str.contains('AMARETTO'), 'liquor_category'] = 'AMARETTO'
df.loc[df['liquor_category'].str.contains('Other') & df['item_description'].str.contains('BOURBON'), 'liquor_category'] = 'WHISKY'
df.loc[df['liquor_category'].str.contains('Other') & df['item_description'].str.contains('LIQUEURS'), 'liquor_category'] = 'LIQUEUR'
df.loc[df['liquor_category'].str.contains('Other') & df['item_description'].str.contains('LIQUEUR'), 'liquor_category'] = 'LIQUEUR'
df.loc[df['liquor_category'].str.contains('Other') & df['item_description'].str.contains('VODKAS'), 'liquor_category'] = 'VODKA'
df.loc[df['liquor_category'].str.contains('Other') & df['item_description'].str.contains('VODKA'), 'liquor_category'] = 'VODKA'
df.loc[df['liquor_category'].str.contains('Other') & df['item_description'].str.contains('BRANDY'), 'liquor_category'] = 'BRANDY'
df.loc[df['liquor_category'].str.contains('Other') & df['item_description'].str.contains('BRANDIES'), 'liquor_category'] = 'GIN'
df.loc[df['liquor_category'].str.contains('Other') & df['item_description'].str.contains('CREME'), 'liquor_category'] = 'SCHNAPPS'
df.loc[df['liquor_category'].str.contains('Other') & df['item_description'].str.contains('SCHNAPPS'), 'liquor_category'] = 'SCHNAPPS'
df.loc[df['liquor_category'].str.contains('Other') & df['item_description'].str.contains('TEQUILA'), 'liquor_category'] = 'TEQUILA'
df.loc[df['liquor_category'].str.contains('Other') & df['item_description'].str.contains('TEQUILAS'), 'liquor_category'] = 'TEQUILA'
df.loc[df['liquor_category'].str.contains('Other') & df['item_description'].str.contains('JOSE'), 'liquor_category'] = 'TEQUILA'
df.loc[df['liquor_category'].str.contains('Other') & df['item_description'].str.contains('PATRON'), 'liquor_category'] = 'TEQUILA'
df.loc[df['liquor_category'].str.contains('Other') & df['item_description'].str.contains('JAMESON'), 'liquor_category'] = 'WHISKY'
df.loc[df['liquor_category'].str.contains('Other') & df['item_description'].str.contains('CROWN'), 'liquor_category'] = 'WHISKY'
df.loc[df['liquor_category'].str.contains('Other') & df['item_description'].str.contains('BEAM'), 'liquor_category'] = 'WHISKY'
df.loc[df['liquor_category'].str.contains('Other') & df['item_description'].str.contains('DANIELS'), 'liquor_category'] = 'WHISKY'
df.loc[df['liquor_category'].str.contains('Other') & df['item_description'].str.contains('MORGAN'), 'liquor_category'] = 'RUM'
df.loc[df['liquor_category'].str.contains('Other') & df['item_description'].str.contains('JOHNNIE WALKER'), 'liquor_category'] = 'WHISKY'
df.loc[df['liquor_category'].str.contains('Other') & df['item_description'].str.contains('GENTLEMAN'), 'liquor_category'] = 'WHISKY'
df.loc[df['liquor_category'].str.contains('Other') & df['item_description'].str.contains('1800'), 'liquor_category'] = 'TEQUILA'
df.loc[df['liquor_category'].str.contains('Other') & df['item_description'].str.contains('BUSHMIL'), 'liquor_category'] = 'WHISKY'
df.loc[df['liquor_category'].str.contains('Other') & df['item_description'].str.contains('KNOB'), 'liquor_category'] = 'WHISKY'
df.loc[df['liquor_category'].str.contains('Other') & df['item_description'].str.contains('TORTILLA'), 'liquor_category'] = 'TEQUILA'
df.loc[df['liquor_category'].str.contains('Other') & df['item_description'].str.contains('GOOSE'), 'liquor_category'] = 'VODKA'
df.loc[df['liquor_category'].str.contains('Other') & df['item_description'].str.contains('MARGARITA'), 'liquor_category'] = 'TEQUILA'
df.loc[df['liquor_category'].str.contains('Other') & df['item_description'].str.contains('TRIPLE SEC'), 'liquor_category'] = 'LIQUEUR'

In [0]:
#upper case category name for matching later
df['store_name'] = df['store_name'].str.upper()

#df['store_name'][40:45]

In [0]:
#grouping all the store locations into parent stores
df['store_parent'] = 'Other'
df.loc[df['store_name'].str.contains('HY-VEE'), 'store_parent'] = 'Hy-Vee'
df.loc[df['store_name'].str.contains('WAL-MART'), 'store_parent'] = 'Wal-Mart'
df.loc[df['store_name'].str.contains('SAM'), 'store_parent'] = 'SamsClub'
df.loc[df['store_name'].str.contains('Fareway'), 'store_parent'] = 'Fareway'
df.loc[df['store_name'].str.contains('KUM'), 'store_parent'] = 'Kum&Go'
df.loc[df['store_name'].str.contains('CVS'), 'store_parent'] = 'CVS'
df.loc[df['store_name'].str.contains('TARGET'), 'store_parent'] = 'Target'
df.loc[df['store_name'].str.contains('CASEY'), 'store_parent'] = 'Caseys'
df.loc[df['store_name'].str.contains('DAHL'), 'store_parent'] = 'Dahls'
df.loc[df['store_name'].str.contains('QUIK'), 'store_parent'] = 'QuikTrip'
df.loc[df['store_name'].str.contains('WALGREEN'), 'store_parent'] = 'Walgreens'
df.loc[df['store_name'].str.contains('SMOKIN'), 'store_parent'] = 'SmokingJoes'
#labored conversion of dates to month year and month year by converting to strings
df['month'] = pd.DatetimeIndex(df['date']).month
df['year'] = pd.DatetimeIndex(df['date']).year

df.loc[df['month'] ==1 , 'month'] = 'Jan'
df.loc[df['month'] ==2 , 'month'] = 'Feb'
df.loc[df['month'] ==3 , 'month'] = 'Mar'
df.loc[df['month'] ==4 , 'month'] = 'Apr'
df.loc[df['month'] ==5 , 'month'] = 'May'
df.loc[df['month'] ==6 , 'month'] = 'Jun'
df.loc[df['month'] ==7 , 'month'] = 'Jul'
df.loc[df['month'] ==8 , 'month'] = 'Aug'
df.loc[df['month'] ==9 , 'month'] = 'Sep'
df.loc[df['month'] ==10 , 'month'] = 'Oct'
df.loc[df['month'] ==11 , 'month'] = 'Nov'
df.loc[df['month'] ==12 , 'month'] = 'Dec'


df.loc[df['year'] ==2012 , 'year'] = '2012'
df.loc[df['year'] ==2013 , 'year'] = '2013'
df.loc[df['year'] ==2014 , 'year'] = '2014'
df.loc[df['year'] ==2015 , 'year'] = '2015'
df.loc[df['year'] ==2016 , 'year'] = '2016'
df.loc[df['year'] ==2017 , 'year'] = '2017'
df.loc[df['year'] ==2018 , 'year'] = '2018'
df.loc[df['year'] ==2019 , 'year'] = '2019'

#merge year and month together
df['monthyear'] = df['month'] + "-" + df['year']

In [0]:
df.head()

At the far right of the above dataset, you can see the 5 new columns we created and thier respective values. 

In [0]:
# replace '?' with -1, we will deal with missing values later
df = df.replace(to_replace='?',value=-1) 

categorical_features = ['city', 'county',  'category_name','vendor_name', 'store_number', 'item_description','store_parent', 
                        'monthyear', 'liquor_category', 'county_number', 'vendor_number', 'item_number']

# let's start by first changing the numeric values to be floats
continuous_features = ['state_bottle_cost', 'state_bottle_retail', 'sale_dollars', 'volume_sold_liters', 'volume_sold_gallons', 'bottles_sold']

# and the oridnal values to be integers
ordinal_features = ['pack', 'bottle_volume_ml']

# use the "astype" function to change the variable type
df[continuous_features] = df[continuous_features].astype(np.float64)
df[ordinal_features] = df[ordinal_features].astype(np.int64)
df[categorical_features] = df[categorical_features].astype(object)
df.info() # now our data looks better!!

In [0]:
# will get summary of continuous or the nominals
dfstats = df.describe()

# let's set those values to NaN, so that Pandas understand they are missing
df = df.replace(to_replace=-1,value=np.nan) # replace -1 with NaN (not a number)
#print (df.info())
df[categorical_features] = df[categorical_features].astype(object)
dfstats2 = df.describe() # scroll over to see the values

categorical_features = ['city', 'county',  'category_name','vendor_name', 'store_number', 'item_description','store_parent', 
                        'monthyear', 'liquor_category', 'county_number', 'vendor_number', 'item_number']

# let's start by first changing the numeric values to be floats
continuous_features = ['state_bottle_cost', 'state_bottle_retail', 'sale_dollars', 'volume_sold_liters', 'volume_sold_gallons', 'bottles_sold']

# and the oridnal values to be integers
ordinal_features = ['pack', 'bottle_volume_ml']

# use the "astype" function to change the variable type
df[continuous_features] = df[continuous_features].astype(np.float64)
df[ordinal_features] = df[ordinal_features].astype(np.int64)
df[categorical_features] = df[categorical_features].astype(object)
df['store_number'] = df['store_number'].astype(object)
df.info() # now our data looks better!!

Here, you can see that our cleaning has addressed all nulls down to zero so we can begin analysis. The count code below helps to show that there is zero nulls in our dataset.

In [0]:
dfna = df[df.isna().any(axis=1)]
dfna.isna().sum()

 ### Check for duplicates

 No Duplicates are present

In [0]:
duplicateRowsDF = df[df.duplicated()]
 
print("Duplicate Rows except first occurrence based on all columns are :")
duplicateRowsDF

### Validate Categorical Features

The next task will be for us to look into our categorical features with over 30 unique values, and we will be dropping any columns that have more than 30 uniques. The reason we are doing this is that we will ultimately have trouble with our predictions as these values will likely ultimately have unique attributes to a specific combination of observations. 

In [0]:
#Unique Value Threshold (Per Column)
#Delete Columns >  uniqueThreshold unique values prior to one-hot encoding. 
#(each unique value becomes a new column during one-hot encoding)
uniqueThreshold = 30
# in each column of dataframe
uniqueValues = df.nunique()
uniqueValues

In [0]:
#Isolate continuous and categorical data types
#These are indexers into the schoolData dataframe and may be used similar to the schoolData dataframe 
D_boolean = df.loc[:, (df.dtypes == bool) ]
D_nominal = df.loc[:, (df.dtypes == object)]
D_continuous = df.loc[:, (df.dtypes != bool) & (df.dtypes != object)]
print ("Boolean Columns: ", D_boolean.shape[1])
print ("Nominal Columns: ", D_nominal.shape[1])
print ("Continuous Columns: ", D_continuous.shape[1])
print ("Columns Accounted for: ", D_nominal.shape[1] + D_continuous.shape[1] + D_boolean.shape[1])

### One Hot Encoding of Categorical Features

In [0]:
#Delete categorical columns with > 30 unique values (Each unique value becomes a column during one-hot encoding)
oneHotUniqueValueCounts = df[D_nominal.columns].apply(lambda x: x.nunique())
oneHotUniqueValueCols = oneHotUniqueValueCounts[oneHotUniqueValueCounts >= uniqueThreshold].index

In [0]:
oneHotUniqueValueCounts
oneHotUniqueValueCols

In [0]:
oneHotUniqueValueCols

In [0]:
onehotlist = oneHotUniqueValueCols.tolist()
onehotlist

In [0]:
#one hot encoding
dfenc = df.copy()
dfenc.head()

In [0]:
dfenc.drop(onehotlist, axis=1, inplace=True)

In [0]:
dfenc.head()

In [0]:
#'month', 'year', 'county_number',
#Isolate continuous and categorical data types
#These are indexers into the schoolData dataframe and may be used similar to the schoolData dataframe 
dfenc_boolean = dfenc.loc[:, (dfenc.dtypes == bool) ]
dfenc_nominal = dfenc.loc[:, (dfenc.dtypes == object)]
dfenc_continuous = dfenc.loc[:, (dfenc.dtypes != bool) & (df.dtypes != object)]
print ("Boolean Columns: ", dfenc_boolean.shape[1])
print ("Nominal Columns: ", dfenc_nominal.shape[1])
print ("Continuous Columns: ", dfenc_continuous.shape[1])
print ("Columns Accounted for: ", dfenc_nominal.shape[1] + dfenc_continuous.shape[1] + dfenc_boolean.shape[1])

one_hot_df = pd.concat([pd.get_dummies(dfenc[col],prefix=col) for col in dfenc_nominal.columns], axis=1)
one_hot_df.head()

In [0]:
df1hotmerge = pd.merge(dfenc, one_hot_df, left_index=True, right_index=True)
df1hotmerge.head()

Encoding table has 51 columns 

On the one hot encoding dataset, we are going to drop the colums, vendor_number, item_number, and store_number as they will be redundant to other features in our dataset.

From here, lets see how big our dataset gets when we onehot our categorical variables. 

In [0]:
df[df.isnull().any(axis=1)][df.columns[df.isnull().any()]]

In [0]:
# this python magics will allow plot to be embedded into the notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore', DeprecationWarning)
%matplotlib inline

In [0]:
df['liquor_category'].value_counts()

Looking at a quick count of our categories, we can see that Vodka appears the most often in our dataet, at over 150k times followed by Rum with over 95k instances. 

Below, we created a column called cost_per_liter and isolated the sales that exceeded $30,000. There were 9 total values with the majority being Vodka varieties. However, looking at cost per liter for each, we can see that these sales were driven more by volume rather than by expensiveness of the alcohol.

In [0]:
# selecting rows based on condition
exp_df = df[df['sale_dollars'] > 30000]

exp_df['cost_per_liter'] = exp_df['sale_dollars']/exp_df['volume_sold_liters']

exp_df[['sale_dollars', 'volume_sold_liters', 'cost_per_liter', 'liquor_category', 'store_parent']]

### Addressing Outliers  
As we have seen, some of these alcohol sales are ranging into the tens of thousands of dollars, which would suggest there are outliers int his dataset. To look into this further we are going to review the sales_dollars feature, cost_per_liter feature, state_bottle_cost feature, bottles_sold and volume_sold features. Firstly, running a box plot with sales dollars, we can't really see the boxes due to how the outliers are ranging up to $50,000. To address this we will need to do a log transform on the continuous features with outliers.

In [0]:
# display boxplot of sale_dollars grouped by liquor_category
plt.style.use('ggplot')
ax = df.boxplot(column = 'sale_dollars', by = 'liquor_category', figsize=(14,7))
plt.xticks(rotation=45)
ax

This box plot is graphing the distribution sale_dollars with the different categories of liquor we have available. Here, we can see there is a significant number of outliers across all categories, so much that you can't really see the boxes on each category. This would suggest we would want to look into a transform.

In [0]:
# perform log transformation on sale_dollars due to significant outliers
df['sale_dollars_trans'] = np.log(df['sale_dollars'])

# display boxplot of logged sale dollars grouped by liquor_category
gx = df.boxplot(column = 'sale_dollars_trans', by = 'liquor_category', figsize=(14,7))
plt.xticks(rotation=45)
gx

This is much more helpful for us to see how our distribution of categories compares to one another. Vodka and Whiskey are still showing a wider range of outliers, however Liquer tends to have a wider bounding than the other categories, which suggests that it has a wide variety of pricing. On average, tequila looks to have the higher medium sale value, which suggest that more money is being spent on tequilla on average. 

Using Sales dollars trans will help address outliers in the future.

In [0]:
# create new feature of cost_per_liter
df['cost_per_liter'] = df['sale_dollars']/df['volume_sold_liters']

# display boxplot of cost_per_liter grouped by liquor_category
bx = df.boxplot(column = 'cost_per_liter', by = 'liquor_category', figsize=(14,7))
plt.xticks(rotation=45)
bx

Looking at the cost per liter aggregate column. As we discussed previously, cost per liter is calculated via a calculation of total sales_dollars divided by the total volume_sold_ml. The idea of this calculation is to see the varying price of the liquors if we normalize by the volume and sale of the liquors in our set. 

Here, we can see that our distribution is right skewed with a large outlier for whiskey which is going for over 1750 per liter. Looking further into this datapoint, we can see that this datapoint represents Johnnie Walker Blue. 

That said, with the distribution of alcohol types, outside of Johnnie walker blue, we cans see that our cost per liter is going to have a greater than tenfold range, so, i would suggest we create a transform there as well. 

In [0]:
df['cost_per_liter_trans'] = np.log(df['cost_per_liter'])

zx = df.boxplot(column = 'cost_per_liter_trans', by = 'liquor_category', figsize=(14,7))
plt.xticks(rotation=45)
zx

This new box plot shows some interesting insight into the distribution and the value placed on alcohol in our dataset. 

Gin and Tequilla - even with outliers, it is going to have a faily consistent price per liter. 
Liquer - Similar case as GIn, however, it's outliers go a little farther out. 
Other- as expected, has a very wide range with very few outliers. Since this represents all types of alcohol not covered in the other categories, the box plot shape is expected. 

Whiskey - Has a faily consistent bounding, however the number out outliers on either side is very large and larger than the other categories. 

In [0]:
#['pack', 'bottle_volume_ml', 'state_bottle_cost', 'bottles_sold', 'sale_dollars_trans', 'volume_sold_liters']

# display boxplot of sale_dollars grouped by liquor_category
plt.style.use('ggplot')
bx = df.boxplot(column = 'state_bottle_cost', by = 'liquor_category', figsize=(14,7))
plt.xticks(rotation=45)
bx



In [0]:
df['state_bottle_cost_trans'] = np.log(df['state_bottle_cost'])

plt.style.use('ggplot')
cx = df.boxplot(column = 'state_bottle_cost_trans', by = 'liquor_category', figsize=(14,7))
plt.xticks(rotation=45)
cx

Looking at state bottle cost, we saw some serious skewing that occured due to outliers, so we decided to transofrm the state bottle cost by the log of the value to help normalize the distribution. Doing so helped to make our box plots cleaner and easier to analyze. 

Looking at the state bottle cost distribution, we can see there is a similar case with skewing that we would need to transform by the log and create a new category called state_bottle_cost_trans.

Next we will do the same thing with the bottles_sold category, which appears to have outliers as well.

In [0]:
# display boxplot of sale_dollars grouped by liquor_category
plt.style.use('ggplot')
dx = df.boxplot(column = 'bottles_sold', by = 'liquor_category', figsize=(14,7))
plt.xticks(rotation=45)
dx

In [0]:
df['bottles_sold_trans'] = np.log(df['bottles_sold'])

plt.style.use('ggplot')
cx = df.boxplot(column = 'bottles_sold_trans', by = 'liquor_category', figsize=(14,7))
plt.xticks(rotation=45)
cx

Next we are going to look at Volume Sold Liters and how that is distributed among our categories. looking at the relationships, we can see there are some outliers that need to be addressed which we will do via a log function. 

In [0]:
# display boxplot of voume sold grouped by liquor_category
plt.style.use('ggplot')
ex = df.boxplot(column = 'volume_sold_liters', by = 'liquor_category', figsize=(14,7))
plt.xticks(rotation=45)
ex

In [0]:
df['volume_sold_liters_trans'] = np.log(df['volume_sold_liters'])

plt.style.use('ggplot')
fx = df.boxplot(column = 'volume_sold_liters_trans', by = 'liquor_category', figsize=(14,7))
plt.xticks(rotation=45)
fx

Now this will be much easier for us to work with. 

## Simple Statistics

The df.describe funciton below provides some simple statistcs for the numeric values in the table. 

Reviewing the 'pack' & 'bottle_volume_ml' variable statistics, we can see that the average transactional sale was about a 12 pack with 960 ml per bottle.  Furthermore, reviewing the 'state_bottle_cost' and 'state_bottle_retail' variables the average cost per bottle is about 10.22 while the average retail price is 15.35. With that said, this goes to align with what a typical box of liquor bottles tend to look like (at 12 per pack) in which a $5 profit is anticipated per bottle.

Another point to consider is the median values are smaller than the mean values. This suggests that our distribution of our sample population is right skewed. 

Lastly, there are a number of outliers within the dataset in which we may need to perform a log tranformation to address.  For example, the 'sale_dollars' variable has max sale amount of 50,186 but the 75th quartile is only at 152.  Also, within the 'volume_sold_liters" variable the max amount is 6,615 liters but the 75th quartile is only 10.5 liters.  Further updats will need to be done to identify and address the outliers within our model.


In [0]:
df.describe()

To further view and analyze the outliers, the scatter plot below easily shows the sales dollars against volume sold by liters (normalized for outliers).  Also, the table that follows shows the top 10 items listed within sale_dollars and volume_sold liters.

In [0]:
# display scatterplot of x=volume_sold_liters, y= sale_dollars
plt.style.use('ggplot')
sns.scatterplot(x=df.volume_sold_liters_trans, y=df.sale_dollars_trans)

In [0]:
# top 10 table for sale_dollars
df.nlargest(10, 'sale_dollars')['item_description'].reset_index()

In [0]:
# top 10 table for volume_sold_liters
df.nlargest(10, 'volume_sold_liters')['item_description'].reset_index()

Some additional interesting facts can be found in the mode section. Vodka has the largest number of sales, and the small package stores combined take up more sales than a single big-box retailer. 

The most common pack size is 12 pack, which is efficient for shipping and the most common bottle size is 750 ml. 

December 2019 is the most popular month for alcohol sales, which suggest that people drink heavily that time of year. :)

In [0]:
dfenc.mode()

## Visualize Attributes
The following section shows our cross tabulations of the relationships we found with our 2019 liquor data. The bar charts and box plots are able to show the distribution of our data and a comparative between categorical variables in our set. 



In [0]:
df['cost_per_liter'] = df['sale_dollars']/df['volume_sold_liters']
df['grossmargin'] = 1 - df['state_bottle_cost']/df['state_bottle_retail']
df_grouped = df.groupby(by=['liquor_category'])

In [0]:
sales_rateq = df_grouped.bottles_sold.sum()
sales_rated = df_grouped.sale_dollars.sum()
sales_ratecpl = df_grouped.cost_per_liter.mean()
salesgm = df_grouped.grossmargin.mean()

In [0]:
dfsr = pd.DataFrame(sales_rateq)
dfsdol = pd.DataFrame(sales_rated)
dfscpl = pd.DataFrame(sales_ratecpl)
dfsg = pd.DataFrame(salesgm)

In [0]:
salesbyliquor = pd.merge(dfsr, dfsdol, how = 'left', on='liquor_category')
salesbyliquor = pd.merge(salesbyliquor, dfscpl, how = 'left', on='liquor_category')
salesbyliquor = pd.merge(salesbyliquor, dfsg, how = 'left', on='liquor_category')
salesbyliquor

In [0]:
salesbyliquor = salesbyliquor.reset_index()

The plots below show various views of the data. The top left shows the grouped average cost per liter by the type of liquor in our dataset. Coming in at over 70 dollars per liter, Liqeuer is the most expensive alcohol in our dataset followed by Whiskey at $45. For Whiskey, some of that can come from the Johnnie Walker Blue we discussed previously, the common theme is that whiskey tends to be more expensive liter for liter than the other types of alcohol.

Looking at an aggregate of our different alcohol categories in the top right chart below, we can see that Vodka generated the highest number of sales in our dataset at close to 2.5 million followed by Rum at closer to 1.5 million. This chart seems to show that Vodka and Rum are the two highest-selling alcohols in the state of Iowa in 2019 with Whiskey coming in 3rd. 

In the bottom right chart the quantity sold by category is shown. While liqeuer had the highest average cost per liter, the vodka tended to have the highest sale quantity per purchase among all the alcohol types. This can be attributed to the volume in which vodka can be made and shipped, which likely affected the volume of sale.

On the bottom left, we can see that the gross margin is consistent among all the categories, which suggest that this could be related to a tax associated with the alcohol sales. We do not think profit is included in this database.

In [0]:
fig, axs = plt.subplots(2, 2, figsize=(18, 18))
sales_rate = salesbyliquor.sort_values(by='sale_dollars', ascending = False)
names = salesbyliquor.sort_values(by='cost_per_liter', ascending=False)['liquor_category'].head(10)
values = salesbyliquor.sort_values(by='cost_per_liter', ascending=False)['cost_per_liter'].head(10)
names2 = salesbyliquor.sort_values(by='sale_dollars', ascending=False)['liquor_category'].head(10)
values2 = salesbyliquor.sort_values(by='sale_dollars', ascending=False)['sale_dollars'].head(10)
names3 = salesbyliquor.sort_values(by='grossmargin',  ascending=False)['liquor_category'].head(10)
values3 = salesbyliquor.sort_values(by='grossmargin',  ascending=False)['grossmargin'].head(10)
names4 = salesbyliquor.sort_values(by='bottles_sold',  ascending=False)['liquor_category'].head(10)
values4 =salesbyliquor.sort_values(by='bottles_sold',  ascending=False)['bottles_sold'].head(10)
axs[0, 0].bar(names, values)
axs[0, 0].set_title('Cost Per Liter')
axs[0, 0].set_xticklabels(names, rotation=20, horizontalalignment='right')
axs[0, 1].bar(names2, values2)
axs[0, 1].set_title('Sales Dollars')
axs[0, 1].set_xticklabels(names2, rotation=20, horizontalalignment='right')
axs[1, 0].bar(names3, values3)
axs[1, 0].set_title('Gross Margin')
axs[1, 0].set_xticklabels(names3, rotation=20, horizontalalignment='right')
axs[1, 1].bar(names4, values4)
axs[1, 1].set_title('Qty Sold')
axs[1, 1].set_xticklabels(names4, rotation=20, horizontalalignment='right')

For our stores in our dataset, as we discussed previously, we decided to focus on the big box retailers as our starting point, and lumped in the smaller stores into a category called "other". As we can see, the other column drove the greatest overall sales at over 2.5 million, however Hy-Vee in Iowa was a close second at 2.3 million in sales. In the state of Iowa, Hy-Vee is a widely popular grocery store chain sells groceries and alocholic beverages.

This same theme is apparent in the QTY sold between retailers in the bottom right. However, interestingly Kum & Go is the retailer with the highest cost per liter. When reviewing the gross margins between the store, our dataset is consistent, which reinforces our hypothesis that this figure represents tax.

In [0]:
df['cost_per_liter'] = df['sale_dollars']/df['volume_sold_liters']
df['grossmargin'] = 1 - df['state_bottle_cost']/df['state_bottle_retail']
df_grouped = df.groupby(by=['store_parent'])

In [0]:
sales_rateq = df_grouped.bottles_sold.sum()
sales_rated = df_grouped.sale_dollars.sum()
sales_ratecpl = df_grouped.cost_per_liter.mean()
salesgm = df_grouped.grossmargin.mean()

In [0]:
dfsr = pd.DataFrame(sales_rateq)
dfsdol = pd.DataFrame(sales_rated)
dfscpl = pd.DataFrame(sales_ratecpl)
dfsg = pd.DataFrame(salesgm)

In [0]:
salesbyparent = pd.merge(dfsr, dfsdol, how = 'left', on='store_parent')
salesbyparent = pd.merge(salesbyparent, dfscpl, how = 'left', on='store_parent')
salesbyparent = pd.merge(salesbyparent, dfsg, how = 'left', on='store_parent')
salesbyparent

In [0]:
salesbyparent = salesbyparent.reset_index()

In [0]:
fig, axs = plt.subplots(2, 2, figsize=(18, 18))
sales_rate = salesbyparent.sort_values(by='sale_dollars', ascending = False)
names = salesbyparent.sort_values(by='cost_per_liter', ascending=False)['store_parent'].head(10)
values = salesbyparent.sort_values(by='cost_per_liter', ascending=False)['cost_per_liter'].head(10)
names2 = salesbyparent.sort_values(by='sale_dollars', ascending=False)['store_parent'].head(10)
values2 = salesbyparent.sort_values(by='sale_dollars', ascending=False)['sale_dollars'].head(10)
names3 = salesbyparent.sort_values(by='grossmargin',  ascending=False)['store_parent'].head(10)
values3 = salesbyparent.sort_values(by='grossmargin',  ascending=False)['grossmargin'].head(10)
names4 = salesbyparent.sort_values(by='bottles_sold',  ascending=False)['store_parent'].head(10)
values4 =salesbyparent.sort_values(by='bottles_sold',  ascending=False)['bottles_sold'].head(10)
axs[0, 0].bar(names, values)
axs[0, 0].set_title('Cost Per Liter')
axs[0, 0].set_xticklabels(names, rotation=20, horizontalalignment='right')
axs[0, 1].bar(names2, values2)
axs[0, 1].set_title('Sales Dollars')
axs[0, 1].set_xticklabels(names2, rotation=20, horizontalalignment='right')
axs[1, 0].bar(names3, values3)
axs[1, 0].set_title('Gross Margin')
axs[1, 0].set_xticklabels(names3, rotation=20, horizontalalignment='right')
axs[1, 1].bar(names4, values4)
axs[1, 1].set_title('Qty Sold')
axs[1, 1].set_xticklabels(names4, rotation=20, horizontalalignment='right')

Looking back at our box plot distribution of transformed sale price, we can see that gin had the narrowest box plot which suggests that the distrobution is slim between sales. Whiskey had the greatest number of outliers, and Vodka appeared to be most right skewed. Suggesting that there were a lot of outliers driving Vodka sales. 

In [0]:
zx = df.boxplot(column = 'cost_per_liter_trans', by = 'liquor_category', figsize=(14,7))
plt.xticks(rotation=45)
zx


### Store Location Analysis
For big box retailers, we focused on Hy-Vee.

In [0]:
hv_df = pd.get_dummies(df['store_parent'], drop_first=False)

hv_df = pd.concat([df, hv_df], axis = 1, sort=False)

In [0]:
hv_df.head()

In [0]:
hv_grouped = hv_df.groupby(by=['liquor_category', 'Hy-Vee']).median()
hv_grouped

Look at Hy-Vee in comparison to not Hy-Vee (all other stores)

Roughly 280K transactions were attributed to stores other than Hy-Vee while roughly 120k transactions were attributed to Hy-Vee.

In [0]:
import seaborn as sns
cmap = sns.diverging_palette(220, 10, as_cmap=True) # one of the many color mappings
sns.countplot(x='Hy-Vee', data=hv_df);

In [0]:
chart=sns.catplot(x='liquor_category', col='Hy-Vee', kind='count', data=hv_df)
chart.set_xticklabels(rotation=45, ha='right')

In the two plots below, we can see the categorical sales tended to remain relatively consistent between Hy-Vee stores as opposed to other liquor stores in the area. This would suggest that Hy-Vee isn't buying liquor differently than anyone else in the state. 

In [0]:
pd.crosstab(hv_df.liquor_category, hv_df['Hy-Vee'], margins=True).style.background_gradient(cmap='autumn_r')

We were curious to see if HyVee Charged Higher Prices than the competitors. Looking at the cost per liter (which would normalize liquor cost), it appears that Hy-Vee had much higher prices.  

In [0]:
sns.catplot('Hy-Vee', "sale_dollars_trans", kind='point', data=hv_df)

In [0]:
sns.catplot('Hy-Vee', "cost_per_liter_trans", kind='point', data=hv_df)

However, grouping by category, we can see that this might have been due to the volume of whiskey sales Hy-Vee had compared to non-hy-vee stores. The costs are pretty consistent.

In [0]:
sns.catplot('Hy-Vee','cost_per_liter_trans',hue='liquor_category', kind='point', data=hv_df);

Now comparing against Small Liquor stores vs large big-box stores, using the same code as above but with Other instead of Hy-Vee.

With 0 being Big Box Stores and 1 Being Small Retailers, we can see that consistently, Small Chains are charging higher prices.

In [0]:
sns.catplot('Other','cost_per_liter_trans',hue='liquor_category', kind='point', data=hv_df);

## Explore Joint Attributes
Visualize relationships between attributes: Look at the attributes via scatter plots, correlation, cross-tabulation, group-wise averages, etc. as appropriate. Explain any interesting relationships.  

The next section dives deeper into the relationsihps between all the variables in our dataset as well as some of the key relationships found between our features. This will be a starting point for us to better understand if we can begin to think about addressing and removing highly correlated features in our set. 

To measure this, we will be able to run grid plots and heat maps with each feature against one another. The plots that are highly coorelated will be closer to 1 while the plots that are highly negatively correlated will be closer to -1. 

Looking at a heat map of our correlation plot, we can see some interesting occurances happening in the bottom right corner with regards to the volume stats as well as the sales stats. We are seeing cases of volume_sold_gallons and liters having a 0.85 correlation to bottles sold. Which makes sense, because the greater the volume of sales would suggest the greater number of bottles in the purchase. 

In addition, these volume stats are highly correlated at 0.85 to sale dollars. While this is expected as it costs more to get more whiskey, it's interesting as there tends to be somewhwat of a minimal affect with regards to volume discounts. 

In [0]:
# plot the correlation matrix using seaborn
# https://towardsdatascience.com/visualizing-data-with-pair-plots-in-python-f228cf529166
import seaborn as sns
cmap = sns.diverging_palette(220, 10, as_cmap=True) # one of the many color mappings

# drop the counter attribute as that was used above when dealing with missing values
del df['counter']

# show the heatmap
sns.set(style="darkgrid") # one of the many styles to plot using
f, ax = plt.subplots(figsize=(18, 9))
chart=sns.heatmap(df.corr(), cmap=cmap, annot=True)
chart.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right')
f.tight_layout()

To explore our joint attributes, we are going to run a pairplot matrix grouped by liquor category across our key continuous variables.  First, we will look at the raw data, but due to skewness we will need to reference the transformed to get a better sense of the story being told.

Here we can see a couple pair scatter plots that show our coorelations in action. In the bottom right corner, we can see those same highly correlated values visualized in a plot, and broken out by drink type.

Interestingly, we can see some clustering start to appear in some of the relationships, particularly between rum, vodka and whiskey as it relates to volume sold and sales data.

In [0]:
sns.pairplot(dfenc, height=3, hue= 'liquor_category', vars = ['state_bottle_retail', 'sale_dollars', 'bottles_sold', 'volume_sold_liters'])

In our transformed pairplot, we can see some slight clustering emerge between our categories and bottles sold, sales and volume sold

In [0]:
sns.pairplot(df, height=3, hue= 'liquor_category', vars = ['state_bottle_retail', 'sale_dollars_trans', 'bottles_sold_trans', 'volume_sold_liters_trans'])

In [0]:
sns.pairplot(df, height=3, hue= 'liquor_category', vars = ['cost_per_liter_trans', 'sale_dollars_trans'])

Next, we are going to look at a cross tab and explore the relationships with alcohol type and store.

As you can see in the table above, Hy Vee and Other take up the majority of sales, however Other is a stronger Vodka seller while Hyvee sales more whieksy. This is confirmed below with our chart that shows the mix of alcohol sales int he story. For each, Vodka and whiskey are among the top sellers. 

In [0]:
#cross tab example
dfcross = pd.crosstab(df['liquor_category'], df['store_parent'],  margins=True, margins_name="Total")
dfcross

In [0]:
#Normalized Cross Tab
dfcrossnorm = pd.crosstab(df['liquor_category'], df['store_parent'], normalize='columns')
dfcrossnorm

Below is a heatmap showing the concentration of categorical sales around the stores. Vodka is a strong seller in Others followed by Rum and Liqueur 

In [0]:

fig, ax = plt.subplots(figsize=(14,7))
chart=sns.heatmap(pd.crosstab([df['liquor_category']], [df['store_parent']]),
            cmap="YlGnBu", annot=True, cbar=False, fmt='g')
chart.set_xticklabels(chart.get_xticklabels(), rotation=45, horizontalalignment='right')

## Explore Attributes and Class

Refining market to whiskey to gather further information about whiskey vendors. Since our analysis is going to focus strictly on whiskey sales, it will be important to see how the whiskey distribution looks within our dataset.

Our goal of exploring attributes and class was to see if there were any relationships between Whiskey purchases and variables within the dataset.  First we analyzed whether or not Whiskey was purchased against 'store_parents' after we analyzed based on location, by county then city.



In [0]:
explore_df= df.copy()

In [0]:
explore_df['WHISKY_Y_N'] = 0
explore_df.loc[explore_df['liquor_category'].str.contains('WHISK'), 'WHISKY_Y_N'] = 1
explore_df.head(5)

After reviewing the table below, it is interesting to note that when Whiskey is purchased it is in packs less then 10.  Also, the average sale_dollars for a whiskey purchase is much greater than non-whiskey purchases.  This is further pointed in out in the average state_bottle_costs.

Lastly, whiskey purchases tend to have a lower average volume sold then non-whiskey purchases.

In [0]:
explore_df.groupby('WHISKY_Y_N').mean()

Based on the table below, Target, Sams Club, & Hy-vee have the highest means when it comes to selling Whiskey in their stores.  In other words, of all the liquor transactions processed within these stores there is about 14 percent chance it is associated to a Whiskey purchase.

In [0]:
explore_df.groupby('store_parent').mean()

As you can see within the table below most whiskey purchases occur in Polk county, with Linn, Johnson, Scott, and Black Hawk following.

In [0]:
large_county_df = explore_df.groupby('county').sum().reset_index()
large_county_df.nlargest(5, 'WHISKY_Y_N')

To further breakdown the location of whiskey purchases, we can see that Des Moines leads all other cities with Cedar Rapids coming in second.

In [0]:
large_county_df = explore_df.groupby('city').sum().reset_index()
large_county_df.nlargest(5, 'WHISKY_Y_N')

### Whiskey Specific Analysis

The first thing we are going to do is copy the original df into a dfwhiskey dataframe.  After we will be filtering for only the 'WHISKY' liquor category.  Note, the new dfwhiskey datafram should have the same features of the original df except it is only filtered for 'WHISKY' specific transactions. 

In [0]:
# copy original df into dfwhiskey dataframe
dfwhiskey = df.copy()

#filter for only the 'WHISKY' liquor_category
dfwhiskey = dfwhiskey[dfwhiskey['liquor_category']=='WHISKY']

# Here's the head of our dataset so we can see how it looks
dfwhiskey.head()

Next, we are going to do a series of calculations to add new columns to our dataset which provides some insight to the sales data of our whiskies. 

These include the following new features:
- profit = state_bottle_retail / bottles_sold
- totalcost = state_bottle_cost * bottles_sold
- revenue = state_bottle_retail * bottles_sold
- grossmargin = (revenue - totalcost)/revenue

Ultimately, this is going to give us a clear view of the profatability, and the market size of whiskey so we can better analyze and predict within the dataset.

In [0]:
#do some calculations for cost and profit
dfwhiskey['profit'] = dfwhiskey['state_bottle_retail']*dfwhiskey['bottles_sold'] - dfwhiskey['state_bottle_cost']* dfwhiskey['bottles_sold']

In [0]:
dfwhiskey['totalcost'] = dfwhiskey['state_bottle_cost']* dfwhiskey['bottles_sold']

In [0]:
dfwhiskey['revenue'] = dfwhiskey['state_bottle_retail']*dfwhiskey['bottles_sold']

In [0]:
dfwhiskey['grossmargin'] = (dfwhiskey['revenue'] - dfwhiskey['totalcost']) / dfwhiskey['revenue']

Here, we sorted gross margin of sale within our data set to get a clearer view of how margins tend to look within the sales in our dataset. 

As we can see, there is a 33% margin that appears across almost all of our sales when you calculate the percentage of what is remaining from the cost over the revenue. We are thiking that is a standard state tax cost for whiskies in the state of Iowa.

In [0]:
#lets do the transforms we did earlier in the EDA
dfwhiskey['sale_dollars_trans'] = np.log(dfwhiskey['sale_dollars'])
dfwhiskey['cost_per_liter_trans'] = np.log(dfwhiskey['cost_per_liter'])

In [0]:
dfwhiskey.sort_values(by='grossmargin', ascending=False).head(5)

Looking at more grossmargin values, we still see the same 33%

In [0]:
dfwhiskey.head(5)

In [0]:
df_grouped = dfwhiskey.groupby(by=['vendor_name'])
df_grouped.head()

To look into some aggregate sums within our whiskey dataset, we are going to create new varuables for the sum of bottles sold and sale_dollars, as well as the mean of cost per liter and gross margin. 

Also, we will create datagrames for each so we can see how the whiskey's distribute in these aggregate figures.

In [0]:
sales_rateq = df_grouped.bottles_sold.sum()
sales_rated = df_grouped.sale_dollars.sum()
sales_ratecpl = df_grouped.cost_per_liter.mean()
salesgm = df_grouped.grossmargin.mean()

In [0]:
dfsr = pd.DataFrame(sales_rateq)
dfsdol = pd.DataFrame(sales_rated)
dfscpl = pd.DataFrame(sales_ratecpl)
dfsg = pd.DataFrame(salesgm)

In [0]:
dfsr.sort_values(by='bottles_sold', ascending=False).head(10)

Looking at the top 10 bottles sold in the whiskey category, we can see that the number one on the list is Pernod Ricard at over 100k bolttles sold, followed distantly by Dieago Americas at 29,500. 

In [0]:
dfsdol.sort_values(by='sale_dollars', ascending=False).head(10)

As a function of bottles sold, we are seeing a similar distirbution with regards to sale_dollars in our top whiskies. Pernod Richard is nearing 3 million dollars in sales followed by Diageo Americas that is under 2 million dollars in sales. 

In [0]:
dfscpl.sort_values(by='cost_per_liter', ascending=False).head(10)

Interestingly, looking at the high cost per liter vendors, Pactific Edge Wine and Spirits has the highest cost per liter at over 145 dollars, followed by Impex Beverage and Hotaling at just under 125 dollars. 

In [0]:
salesbyvendor = pd.merge(dfsr, dfsdol, how = 'left', on='vendor_name')
salesbyvendor = pd.merge(salesbyvendor, dfscpl, how = 'left', on='vendor_name')
salesbyvendor = pd.merge(salesbyvendor, dfsg, how = 'left', on='vendor_name')
salesbyvendor

Next, we are going to merge our datasets so that we can group our measures of bottles sold, sale_dollars, cost_per_liter and gross margin by our 61 vendors, so we can see how they rank in our key finance values. 

In [0]:
salesbyvendor.sort_values(by='cost_per_liter', ascending=False).head(10)

In [0]:
salesbyvendor.sort_values(by='sale_dollars', ascending=False).head(10)

In [0]:
salesbyvendor.sort_values(by='grossmargin', ascending=False).head(100)

Below are some additional plots to help you visualize a more complete dataset on cost per liter as well as sales dollar of our specific whiskies in the study. As discussed before, the Pernod Richard whiskey is the top selling vendor,while Pacific Edge Whiskey has the highest cost per liter, implying they are seeling the most valuable whiskey.

In [0]:
salesbyvendor = salesbyvendor.reset_index()

fig, axs = plt.subplots(2, 2, figsize=(18, 18))
sales_rate = salesbyvendor.sort_values(by='sale_dollars', ascending = False)
names = salesbyvendor.sort_values(by='cost_per_liter', ascending=False)['vendor_name'].head(10)
values = salesbyvendor.sort_values(by='cost_per_liter', ascending=False)['cost_per_liter'].head(10)
names2 = salesbyvendor.sort_values(by='sale_dollars', ascending=False)['vendor_name'].head(10)
values2 = salesbyvendor.sort_values(by='sale_dollars', ascending=False)['sale_dollars'].head(10)
names3 = salesbyvendor.sort_values(by='grossmargin',  ascending=False)['vendor_name'].head(10)
values3 = salesbyvendor.sort_values(by='grossmargin',  ascending=False)['grossmargin'].head(10)
names4 = salesbyvendor.sort_values(by='bottles_sold',  ascending=False)['vendor_name'].head(10)
values4 =salesbyvendor.sort_values(by='bottles_sold',  ascending=False)['bottles_sold'].head(10)
axs[0, 0].bar(names, values)
axs[0, 0].set_title('Cost Per Liter')
axs[0, 0].set_xticklabels(names, rotation=20, horizontalalignment='right')
axs[0, 1].bar(names2, values2)
axs[0, 1].set_title('Sales Dollars')
axs[0, 1].set_xticklabels(names2, rotation=20, horizontalalignment='right')
axs[1, 0].bar(names3, values3)
axs[1, 0].set_title('Gross Margin')
axs[1, 0].set_xticklabels(names3, rotation=20, horizontalalignment='right')
axs[1, 1].bar(names4, values4)
axs[1, 1].set_title('Qty Sold')
axs[1, 1].set_xticklabels(names4, rotation=20, horizontalalignment='right')

## New Features

Some of the new features that we would like to join in on this data would be census data.  There might be good information correlated between education, population, income groups, age groups, employment that could help classifying which retailers are more likely to sell greater volumes of whiskey.  Because there is so many different types of liquor (over 130 category names), we created a new feature, liquor_category that generalizes the liquor.  This way we can look at whiskey in a general sense against the other liquors.  Also, a new feature that could be created from existing data would be the day of the week from the sale date.  Would use the day of the week to look at sales from a weekday perspective (Monday, Tues...Sat, Sun) perspective.  This way we can aggregate the sales by day of week to help on logistics and prevent stock outs.  Below is the list of new features called out: 
* Day of the week  
* Education  
* Median population  
* Income groups  
* Age groups  
* Employment  
* Grouping of Liquor Brands  

We also created a series of columns which were aggregates of other columns.  These new features were added to provide more insight into the sales data to give better view of the profatability, and the market size of whiskey so we can better analyze and predict within the dataset

* Cost Per Liter
  * cost_per_liter = sale_dollars / volume_sold_liters
  * This helps normalize the costing structure of each of the liquor and have a standard view on pricing.  Since the sale_dollars is based on the number of bottles sold * retail price, we weren't able to compare the cost of liquor effectively
* Total Cost
  * totalcost = state_bottle_cost * bottles_sold
  * Provides total cost of the whiskey and will be used in determining our operating margin
* Revenue 
  * revenue = state_bottle_cost * bottles_sold
  * Need to see how much revenue is being generated.  Basic metric used in running any business
* Gross Margin
  * grossmargin = (revenue - totalcost)/revenue
  * Need to see how profitable we are doing in whiskey sales
* Log Transform of Sales
  * The sale_dollars is heavily right skewed, so performing a log transformation on the data helps normalize the data.
* Log Transform of Volume Sold
  * The volume_sold_liters is heavily right skewed, performed a log transform on this data as well to normalize the data.
* Log Transform of Bottles sold
  * The bottles_sold feature with a log transform applied so that it normalizes the distribution
* Log Transform of Cost Per Liter
  * The Cost_per_liter feature with a log transform applied to normalize the distribution




## Exceptional Work 
Work that we would like to be evaluated as exceptional work would be what we did around the areas of:
 * Tableau Location Analysis
 * Feature Reduction using PCA and LDA as well as dropping highly correlated variables
 * Time series analysis across the entire 2012 to 2019 dataset via big query
 * One hot encoding done on the data 



### Tableau Location Analysis
We built a tableau dashboard out of the 2019 iowa dataset with interative maps of where the most volume of alcohol was sold as well as the a time series analysis of sales and volume by date over 2019. The map features a caption with the analysis of the visuals.

https://public.tableau.com/profile/daniel.clark1522#!/vizhome/WhiskeySaleStory/Story1?publish=yes

In [0]:
### Remove highly correlated features

In [0]:
df_corr = df.copy()

In [0]:
df_corr.head()

In [0]:

# Create correlation matrix
corr_matrix = df_corr.corr().abs()
corr_matrix

In [0]:
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

# courtesy of https://chrisalbon.com/machine_learning/feature_selection/drop_highly_correlated_features/

In [0]:
to_drop

Looking at our correlation matrix, we can deem that per our criteria of removing features that have a greater than 95% correlation. As the predictability of one feature will detirmine another, which will cause us to want to do a dimensionality reduction. 
In our case, we are going to remove state_bottle_retail as it correlates highly with state_bottle_cost, as well as volume_sold_gallons as it correlates perfectly with volume_sold_liters.

In [0]:
# Drop features 
df_corr = df_corr.drop(df_corr[to_drop], axis=1)

### Feature Reduction using PCA and LDA
PCA
To run a PCA analysis, we will want to first remove some of the highly correlated variables which may throw off our analysis. This will mean we should run a correlation plot and set our correlation threshold to 0.95, which we will remove variables if we exceed. 

First lets take a look at our seaborn plot that we will use to run principal component analysis. To get started, we will want to create dummies for each of our categories of alcohol, so that we have a single binary under Whiskey as the sale was a whiskey drink (1) or it was not (0)

In [0]:
df_corr2 = pd.get_dummies(df['liquor_category'], drop_first=False)

df_corr = pd.concat([df_corr, df_corr2], axis = 1, sort=False)



In [0]:
# now let's use PCA, and LDA to find the two "best" dimensions of this data
# these are linear transforms to help project the features into something more understandable

from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

#drop the response variable from x
# one hot encoding to category
features = ['pack', 'bottle_volume_ml', 'state_bottle_cost', 'bottles_sold', 'sale_dollars_trans', 'volume_sold_liters']

X = df_corr.loc[:, features].values
# run this second line below which compiles all the features that i want to use to predict category, or the column we want to predict
#X = X.drop (category, inplace = True)
y = df_corr.loc[:, ['WHISKY']].values
#target_names = df_corr.store_parent_names

pca = PCA(n_components=2)
X_pca = pca.fit(X).transform(X) # fit data and then transform it

lda = LDA(n_components=2)
X_lda = lda.fit(X, y).transform(X) # fit data and then transform it

# print the components

print ('pca:', pca.components_)
print ('lda:', lda.scalings_.T)

Now what we did here was isolate our categorical variable "Whiskey" and fit a pca and LDA model using the continuous variables remaining in the set. Using this, our model will be able to advise on how much we can explain variance using the different principal components. 
Scaling it in this way allows our model to utilize and compare all variables on the same plane. 

In [0]:
principalDf = pd.DataFrame(data = X_pca
             , columns = ['principal component 1', 'principal component 2'])

principalDf.describe()

creating a dataframe of our principal components, we can see that there is quite a range in the values we have, from the negatives to the positives, with also a strong standard deviation. 
The mean principal component is nearly off the scale due to the effect of outliers. 

In [0]:
finalDf = pd.concat([principalDf, df_corr[['WHISKY']]], axis = 1)

finalDf.head()

In [0]:
fig = plt.figure(figsize = (10,10))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)
targets = ['Whiskey', 'Not Whiskey']
colors = ['r', 'g', 'b']
for target, color in zip(targets,colors):
    indicesToKeep = finalDf['WHISKY'] == 0
    ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1']
               , finalDf.loc[indicesToKeep, 'principal component 2']
               , c = color
               , s = 40)
ax.legend(targets)
ax.grid()

Looking at the plot we graphed, we can see the effect of outliers and the fact that non/whiskies have quite a presence in our chart. 
Our explained variance ratio below shows that we can explain 97% of the variance with one principal component, and less than 3% with the second principal component.

In [0]:
pca.explained_variance_ratio_

In [0]:
#lets transform our outliers to make them easier to read. 

finalDf2 = finalDf

finalDf2['principal component 1 trans'] = np.log(finalDf['principal component 1'])
finalDf2['principal component 2 trans'] = np.log(finalDf['principal component 2'])

finalDf2.head()

In [0]:
fig = plt.figure(figsize = (10,10))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)
targets = ['Whiskey', 'Not Whiskey']
colors = ['r', 'g', 'b']
for target, color in zip(targets,colors):
    indicesToKeep = finalDf2['WHISKY'] == 1
    ax.scatter(finalDf2.loc[indicesToKeep, 'principal component 1 trans']
               , finalDf2.loc[indicesToKeep, 'principal component 2 trans']
               , c = color
               , s = 40)
ax.legend(targets)
ax.grid()

To make our chart easier to read, i decided to take a transform of our principal components to normalize their distribution. As you can see, their values separate a lot more in our dataset. However, the non whiskey values play a small presence.

### Time series analysis across the entire 2012 to 2019 dataset via big query

Registered for credentials with Google Cloud and Big Query to be able to query our database and data frame from 17 million datapoints for analysis on our project. 

In our Visualize Attributes section, we went into a deep dive on the analysis of whiskey as well as Hy-Vee to understand their effect on the dataset.

Because the dataset is so large, 17.7 million rows and over 4GB in size, we couldn't pull the entire dataset from BigQuery into Pandas.  

So, initial analysis of the entire dataset was done by querying BigQuery

First needed to connect to BigQuery, then do a some EDA against the data set by querying BigQuery and visualize those results to determine a good subset of the data.

In [0]:
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore', DeprecationWarning)
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
project_id = 'smu-7331-ml'

plt.rc('axes', axisbelow=True)

# Imports the Google Cloud client library
#from google.cloud import bigquery
# Instantiates a client for BigQuery Service
#bqclient = bigquery.Client()

from google.colab import auth
auth.authenticate_user()
print('Authenticated')

from google.cloud import bigquery
bqclient = bigquery.Client(project=project_id)


**Functions**

**Execute Query** 

In [0]:
def execute_query(QUERY):
    # Run the query and get data from BQ
    query_job = bqclient.query(QUERY)
    return query_job.to_dataframe()

**Exploratory Data Analysis**

**Total data size and column types**

In [0]:
QUERY="""
    SELECT
        count(*) as TotalRows
    FROM `bigquery-public-data.iowa_liquor_sales.sales`
"""
df=execute_query(QUERY)
df

**Get Columns and Data Types for Columns**

In [0]:
QUERY="""
    SELECT
        *
    FROM `bigquery-public-data.iowa_liquor_sales.sales`
    LIMIT 10
"""
df=execute_query(QUERY)
df.info()

In [0]:
pd.set_option('display.max_columns', None)
df.head()

Look at distribution of data for each Year-Month

In [0]:
# How many rows of data for each month for each year 
QUERY="""
SELECT
    extract(YEAR from date) as year,
    extract(MONTH from date) as month,
    count(*) as count
FROM `bigquery-public-data.iowa_liquor_sales.sales`
group by year, month
order by year, month
"""
df = execute_query(QUERY)

# Plot the results
df['period'] = df['year'].map(str) +'-' +df['month'].map(str)
#df.groupby(['year', 'month'])['count'].sum().plot(kind='bar', figsize=(20, 8))
plot=df.plot(kind='bar', x='period', y='count', figsize=(20,8))
#xtick = pd.date_range( start=ts.index.min( ), end=ts.index.max( ), freq='W' )
#plot.set_xticks( xtick, minor=True )
#plot.grid('off', which='minor', axis='y' )
plot.grid('on', axis='y', linestyle='dashed')
plt.yticks(np.arange(0, max(df['count']+10000), 20000))
plt.show()

**Determine the different liquor categories**

In [0]:
# How many rows of data for each month for each year 
QUERY="""
SELECT
    category_name,
    count(*) as count
FROM `bigquery-public-data.iowa_liquor_sales.sales`
group by category_name
order by category_name
"""
df = execute_query(QUERY)
df

Group vodka, rum, whiskey, tequila as separate categories, and everything else as other

In [0]:
QUERY="""
SELECT
CASE
  WHEN lower(category_name) like '%rum%' then 'Rum'
  WHEN lower(category_name) like '%whisk%' then 'Whiskey'
  WHEN lower(category_name) like '%vodka%' then 'Vodka'
  WHEN lower(category_name) like '%tequila%' then 'Tequila'
  WHEN lower(category_name) like '%schnapps%' then 'Schnapps'
  WHEN lower(category_name) like '%gin%' then 'Gin'
  WHEN lower(category_name) like '%bourbon%' then 'Bourbon'
  WHEN lower(category_name) like '%brandies%' then 'Brandies'
  WHEN lower(category_name) like '%liqueur%' then 'Liqueur'
  WHEN lower(category_name) like '%scotch%' then 'Scotch'
  ELSE 'Other'
END
as CategoryName,
extract(Year from date) as Year,
extract(month from date) as Month,
count(*) as Count
FROM
  `bigquery-public-data.iowa_liquor_sales.sales`
group by year, month, categoryname
order by year, month, CategoryName
"""
df = execute_query(QUERY)
catYear=df.groupby(['CategoryName', 'Year'])['Count'].sum().unstack().plot(kind='bar', stacked=True, figsize=(14,7))
catYear.grid('on', axis='y', linestyle='dashed')
plt.yticks(np.arange(0, 6000000, 500000))
plt.gca().legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.xticks(rotation=45, horizontalalignment='right')
plt.show()

In [0]:
yearCat=df.groupby(['Year', 'CategoryName'])['Count'].sum().unstack().plot(kind='bar', stacked=True, figsize=(14,7))
yearCat.grid('on', axis='y', linestyle='dashed')
plt.yticks(np.arange(0, 3000000, 500000))
plt.gca().legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.xticks(rotation=45, horizontalalignment='right')
plt.show()

Subset the data

Based on the results above, we decided to go with 400,000 random rows from 2019.

Our challenge was with github only allowing a file smaller than 100mb.  So we found that 400k rows came in at 95mb.

Because there wasn't a particular sort column, we simply extracted 400k rows from BigQuery by executing the following query:


```
SELECT
  *
FROM `bigquery-public-data.iowa_liquor_sales.sales`
WHERE
date between "2019-01-01" and "2019-12-31" limit 400000;
```

Once executed, we were able to save the results (400,000 rows) to a csv, and that is what we are using for our data set


### One Hot Encoding
To be able to put our dataset into a manner that allows us to analyze further, we ran a one hot encoding procedure that isolated the categorical variables and one-hotted the ones that exceeded 30 uniques. This was done in our data cleaning stage earlier in this doc.